In [ ]:
%%capture cap
!pip3 install -U bitsandbytes
!pip3 install -U pef
!pip3 install -U trl
!pip3 install -U accelerate
!pip3 install -U datasets
!pip3 install -U transformers
!pip3 install langchain sentence-transformers chromadb langchainhub



In [ ]:
with open("pip.log", "w") as file:
    file.write(cap.stdout)

In [4]:
from langchain_community.llms import Ollama

In [11]:
# The model we want
model = "gemma:2b-instruct"

# Connect to the Ollama isntance
llm = Ollama(model=model)

# Have a basic prompt
prompt = "Tell me a joke"


In [9]:

# Invoke with a prompt, response as a whole.
print(f"Invoking prompt: '{prompt}' ...")
response = llm.invoke(prompt)
print(response)
llm.invoke("Tell me a joke")

Invoking prompt: 'Tell me a joke' ...


Why did the math book look so sad?

Because of all of its problems!


'Why did the math book look so sad?\n\nBecause of all of its problems.'

In [10]:
chunks = []
print(f"Streaming response for prompt: '{prompt}' ...")
for chunk in llm.stream(prompt):
    chunks.append(chunk)
    print(chunk, end="", flush=False)

Streaming response for prompt: 'Tell me a joke' ...
Why did the math book look so sad?

Because of all of its problems!

## Prompt Template

### One question

In [13]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

question = "Who won the FIFA World Cup in the year 1994?"

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate.from_template(template)

llm_chain = LLMChain(prompt=prompt, llm=llm)
print(llm_chain.invoke({"question": question}))

{'question': 'Who won the FIFA World Cup in the year 1994?', 'text': 'The provided text does not contain any information regarding the FIFA World Cup winner in 1994, so I am unable to answer this question from the provided context.'}


### Multiple Questions

In [14]:
qs = [
    {'question': "What is Kaggle?"},
    {'question': "What is the first step I should take on Kaggle?"},
    {'question': "I followed your instructions. What should I do next?"}
]

res = llm_chain.generate(qs)
print(res.generations)

[[GenerationChunk(text='**Step 1: Understanding the concept of Kaggle.**\n\n- Kaggle is a platform that provides a platform for data science and machine learning communities to collaborate, compete, and share datasets.\n\n\n**Step 2: Defining its purpose.**\n\n- Kaggle is primarily known for hosting competitions and providing access to datasets relevant to various data science and machine learning tasks.\n\n\n**Step 3: Recognizing its significance.**\n\n- Kaggle serves as a valuable resource for data enthusiasts, professionals, and organizations to enhance their skills, explore new ideas, and compete in meaningful challenges.\n\n\n**Step 4: Understanding its key features.**\n\n- Kaggle offers a variety of features, including:\n    - Competitions with predefined datasets and tasks\n    - Datasets categorized by domain and industry\n    - Opportunity to collaborate with other data scientists and learn from their expertise\n    - Access to tools and resources for data exploration, modelin

In [18]:
### With Context Inlined

In [19]:
prompt = """Answer the question based on the context below. If the question cannot be answered using the information provided, answer with "I don't know".

Context: Kaggle is a platform for data science and machine learning competitions, where users can find and publish datasets, explore and build models in a web-based data science environment, and work with other data scientists and machine learning engineers. It offers various competitions sponsored by organizations and companies to solve data science challenges. Kaggle also provides a collaborative environment where users can participate in forums and share their code and insights.

Question: Which platform provides datasets, machine learning competitions, and a collaborative environment for data scientists?

Answer:"""

print(llm.invoke(prompt))

**Platform:** Kaggle


In [21]:
### FewShot (i.e. with some examples)

In [20]:
# Import the FewShotPromptTemplate class from langchain module
from langchain import FewShotPromptTemplate

# Define examples that include user queries and AI's answers specific to Kaggle competitions
examples = [
    {
        "query": "How do I start with Kaggle competitions?",
        "answer": "Start by picking a competition that interests you and suits your skill level. Don't worry about winning; focus on learning and improving your skills."
    },
    {
        "query": "What should I do if my model isn't performing well?",
        "answer": "It's all part of the process! Try exploring different models, tuning your hyperparameters, and don't forget to check the forums for tips from other Kagglers."
    },
    {
        "query": "How can I find a team to join on Kaggle?",
        "answer": "Check out the competition's discussion forums. Many teams look for members there, or you can post your own interest in joining a team. It's a great way to learn from others and share your skills."
    }
]

# Define the format for how each example should be presented in the prompt
example_template = """
User: {query}
AI: {answer}
"""

# Create an instance of PromptTemplate for formatting the examples
example_prompt = PromptTemplate(
    input_variables=['query', 'answer'],
    template=example_template
)

# Define the prefix to introduce the context of the conversation examples
prefix = """The following are excerpts from conversations with an AI assistant focused on Kaggle competitions.
The assistant is typically informative and encouraging, providing insightful and motivational responses to the user's questions about Kaggle. Here are some examples:
"""

# Define the suffix that specifies the format for presenting the new query to the AI
suffix = """
User: {query}
AI: """

# Create an instance of FewShotPromptTemplate with the defined examples, templates, and formatting
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

query = "Is participating in Kaggle competitions worth my time?"

print(llm.invoke(few_shot_prompt_template.format(query=query)))

**Overall Tone:** Informative, encouraging, and supportive.

**Specific Responses:**

**How do I start with Kaggle competitions?**
- Start by selecting a competition that resonates with you and aligns with your skill level.

**What should I do if my model isn't performing well?**
- Explore various models, refine hyperparameters, and consult forums for insights.

**How can I find a team to join on Kaggle?**
- Explore competition discussion forums to identify potential team members.
- Post your interest in joining a team.

**Is participating in Kaggle competitions worth my time?**
- Yes, participating in Kaggle competitions offers valuable learning and networking opportunities.
